In [2]:
import os,sys
import numpy as np
# xarray
import xarray as xr

# import zarr

In [3]:
from dask.distributed import Client
c = Client()
c

/linkhome/rech/genlop01/uor98hu/.local/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43607 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43607/status,
Dashboard: http://127.0.0.1:43607/status,Workers: 8
Total threads: 40,Total memory: 80.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43355,Workers: 8
Dashboard: http://127.0.0.1:43607/status,Total threads: 40
Started: Just now,Total memory: 80.00 GiB
Comm: tcp://127.0.0.1:44507,Total threads: 5
Dashboard: http://127.0.0.1:39295/status,Memory: 10.00 GiB
Nanny: tcp://127.0.0.1:34513,


In [4]:
# ------------------------
# load the mask of interest
# ------------------------
# maskBG = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/TRANSPORT_WP1/BGRmaskobs.nc', chunks = {"x":200,"y":320}).BGRmask.squeeze()
dirgr = "/gpfswork/rech/cli/uor98hu/MYDATA/remoteGr_mask/"
# amerasian mask
maskAme = xr.open_dataset(dirgr+'/Arctic_Amerasian_mask.nc', chunks = {"x":200,"y":320}).polymask.squeeze()

# Arctic NOGr
Arctic_westnogr_mask = xr.open_dataset(dirgr + 'Arctic_westNoGr_mask.nc',chunks = {"x":200,"y":320}).polymask.squeeze()
Arctic_estnogr_mask = xr.open_dataset(dirgr+'Arctic_estNoGr_mask.nc',chunks = {"x":200,"y":320}).polymask.squeeze()
Arctic_nogr_mask = Arctic_westnogr_mask + Arctic_estnogr_mask

# arctic no greenland est
Arctic_west_mask = xr.open_dataset(dirgr+'Arctic_west_mask.nc',chunks = {"x":200,"y":320}).polymask.squeeze()
Arctic_est_maskN = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/remoteGr_mask/Arctic_est_mask.nc',chunks = {"x":200,"y":320}).polymask.squeeze()
Arctic_NoGrEst_mask = Arctic_est_maskN + Arctic_west_mask

# arctic full
Arctic_est_mask = xr.open_dataset(dirgr+'Arctic_est_GrSv2_mask.nc',chunks = {"x":200,"y":320}).polymask.squeeze()
Arctic_mask = Arctic_west_mask + Arctic_est_mask

list_masks = [maskAme.load(), Arctic_nogr_mask.load(), Arctic_NoGrEst_mask.load(), Arctic_mask.load()]
list_mask_name = ["amerasian","no_gr", "no_gr_est", "full_arctic"]

In [5]:
%%time
# ------------------------
# loading data to prepare for the volume weighted mean: e1t, e2t, e3t
# ------------------------
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr=xr.open_dataset(diri+'mesh_hgr.nc').squeeze() #  chunks = {"x":200,"y":320}
e1t = mesh_hgr.e1t.fillna(0)
e2t = mesh_hgr.e2t.fillna(0)

prefix = "eORCA025.L75-IMHOTEP"
dirww = "/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/data4figure/final_after_review/"

nbsec  = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/nbsec_1m.nc').nbsec # chunks={'time_counter':1}

# les lignes à modifier entre expériences
nexpREF = "GAI"
dirsc = "/gpfsscratch/rech/cli/uor98hu/BILANS/"+nexpREF+"/dSdt/gpfsscratch/rech/cli/uor98hu/BILANS/GAI/dSdt/"

# nexpREF = "AI"
# dirsc = "/gpfsscratch/rech/cli/uor98hu/BILANS/"+nexpREF+"/"

# nexpREF = "S"
# dirsc = "/gpfsscratch/rech/cli/uor98hu/BILANS/"+nexpREF+"/"
# fin lignes à modifier

diridatref="/gpfsstore/rech/cli/rcli002/eORCA025.L75/"+prefix+"."+nexpREF+"-S/"
chunk_size = {"x":200,"y":320, 'deptht':40}

e3t = xr.open_mfdataset(diridatref+"1m/????/*1m_gridT.nc",decode_coords=False,chunks=chunk_size\
                            ,parallel=True).e3t

# ------------------------
# load dS/dt
# ------------------------
dSdt = xr.open_mfdataset(dirsc+nexpREF+"_dSdt_1m*.nc", parallel=True, chunks={"x":200,"y":320,'deptht':40}).dSdt.squeeze()

CPU times: user 11.1 s, sys: 1.08 s, total: 12.2 s
Wall time: 23.8 s


In [6]:
dSdt

<xarray.DataArray 'dSdt' (time_counter: 468, deptht: 75, y: 1207, x: 1442)>
dask.array<concatenate, shape=(468, 75, 1207, 1442), dtype=float64, chunksize=(12, 40, 320, 200), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 1980-01-16T12:00:00 ... 2018-...
  * deptht        (deptht) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    nav_lat       (y, x) float32 dask.array<chunksize=(320, 200), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(320, 200), meta=np.ndarray>
Dimensions without coordinates: y, x

In [14]:
bgr_e3t
btbox

<xarray.DataArray (time_counter: 468, deptht: 75, y: 191, x: 786)>
dask.array<mul, shape=(468, 75, 191, 786), dtype=float64, chunksize=(1, 40, 191, 200), chunktype=numpy.ndarray>
Coordinates:
  * deptht        (deptht) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
  * time_counter  (time_counter) datetime64[ns] 1980-01-16T12:00:00 ... 2018-...
Dimensions without coordinates: y, x

In [15]:
%%time

# for i in np.arange(1,4):
i=1
# ------------------------
# compute the volume weight
# ------------------------
bgr_e1t = e1t.where(list_masks[i],drop=True)
bgr_e2t = e2t.where(list_masks[i],drop=True)

bgr_e3t = e3t.where(list_masks[i],drop=True)
bgr_e3t = e3t.where(list_masks[i],drop=True) # chunk({'time_counter':e3t.shape[0]})

# gridcell volume
btbox = ( bgr_e3t*bgr_e2t*bgr_e1t ) #.compute()
voltotal = btbox.sum(("deptht","y","x"))

# ------------------------
# compute the Gtotal term
# ------------------------
bgr_dSdt = dSdt.where(list_masks[i],drop=True)
bgr_dSdt_mean = (bgr_dSdt * btbox).sum(("deptht","y","x")) / voltotal


bgr_dSdt_mean_comp = bgr_dSdt_mean.compute()
bgr_dS_mean_compcumsum = (bgr_dSdt_mean_comp*nbsec).cumsum().compute()

# ----------------
# Save
# ----------------
bgr_dS_mean_compcumsum.to_dataset(name='dScumsum').to_netcdf(path = dirww+list_mask_name[i]+"/"+nexpREF+"_dScumsum_1980_2018_after_review.nc", mode='w')

2024-02-12 12:26:52,975 - distributed.nanny - WARNING - Restarting worker
2024-02-12 12:28:18,676 - distributed.nanny - WARNING - Restarting worker
2024-02-12 12:29:00,989 - distributed.nanny - WARNING - Restarting worker
2024-02-12 12:32:01,607 - distributed.nanny - WARNING - Restarting worker


KilledWorker: Attempted to run task ('concatenate-open_dataset-getitem-5b97268905695dd770fd14f9687c62bc', 386, 1, 0, 1) on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:44507. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.